In [7]:
# one way to create coroutines is by using the coroutine decorator from the
# types module

from types import coroutine

@coroutine
def nothing():
    yield 'yielded from nothing'
    return ('returned from nothing')

@coroutine
def count(num):
    for i in range(num):
        yield f'from the inner loop - count: {i}'
    return 'returned from count'

async def do_a_few_things(num=3, name='no_name'):
    for i in range(num):
        print(f'in the {name} loop (middle loop) for the {i}th time')
        from_await = await count(i + 2)
        print('value returned from await:', from_await)

# we're going to create a class to make a task loop
class TaskLoop():
    def __init__(self):
        # list to hold the tasks
        self.tasks = []

    def add_task(self, task):
        # add a task to the loop task must be a coroutine
        self.tasks.append(task)

    def run_all(self):
        # this is where the task loop runs
        # keep a loop going until all the tasks are gone
        i = 0
        while self.tasks:
            i += 1
            print(f'\nOuter loop count: {i}')
            # pop a task off the end
            task = self.tasks.pop()
            # run that task
            try:
                res = task.send(None)
                print('result of send:', res)
                #put it back on the beginning of the task list
                self.tasks.insert(0, task)
            except StopIteration:
                # this will be raised if it is done
                # so we don't put it back on the task list
                print('The awaitable is complete')

print('\n\n*** Running the Loop class\n')

# to use it we create a task loop object and add tasks to it
loop = TaskLoop()
loop.add_task(do_a_few_things(2, 'first task'))
loop.add_task(do_a_few_things(4, 'second task'))
loop.add_task(do_a_few_things(3, 'third task'))

# and then call run_all
loop.run_all()




*** Running the Loop class


Outer loop count: 1
in the third task loop (middle loop) for the 0th time
result of send: from the inner loop - count: 0

Outer loop count: 2
in the second task loop (middle loop) for the 0th time
result of send: from the inner loop - count: 0

Outer loop count: 3
in the first task loop (middle loop) for the 0th time
result of send: from the inner loop - count: 0

Outer loop count: 4
result of send: from the inner loop - count: 1

Outer loop count: 5
result of send: from the inner loop - count: 1

Outer loop count: 6
result of send: from the inner loop - count: 1

Outer loop count: 7
value returned from await: returned from count
in the third task loop (middle loop) for the 1th time
result of send: from the inner loop - count: 0

Outer loop count: 8
value returned from await: returned from count
in the second task loop (middle loop) for the 1th time
result of send: from the inner loop - count: 0

Outer loop count: 9
value returned from await: returned fro

In [18]:
# let's add some tasks that actually take 'time' to complete simulating what would happen if a task were waiting
# on say a network request or some I/O function to take place/etc.
# note - we can't use the 'sleep' command because that is blocking instead we will...

import time
from types import coroutine

@coroutine
def sleep(secs=0):
    start = time.time()
    # now we need it to yield control
    yield '{} seconds have passed'.format(time.time() - start)
    # and keep yielding it until  enough time has passed
    while time.time() - start < secs:
        # now we need it to yield control
        yield 'yielding in sleep'
    return '{} seconds have passed'.format(time.time() - start)

# now we'll create a coroutine that calculates something

async def fib(n):
    # classic fibbonacci number, but with a delay
    if n == 0:
        return 0
    a, b = 0, 1
    for i in range(n - 1):
        a, b = b, a + b
    # await sleep(0.0)
    return b    

# we're going to create a class to make a task loop
class TaskLoop():
    def __init__(self):
        # list to hold the tasks
        self.tasks = []

    def add_task(self, task):
        # add a task to the loop task must be a coroutine
        self.tasks.append(task)

    def run_all(self):
        # this is where the task loop runs
        # list to hold the results
        results = []
        # keep a loop going until all the tasks are gone
        i = 0
        while self.tasks:
            i += 1
            time.sleep(0.001)
            print(f'\nOuter loop count: {i}')
            # pop a task off the end
            task = self.tasks.pop()
            # run that task
            try:
                res = task.send(None)
                print('result of send:', res)
                #put it back on the beginning of the task list
                self.tasks.insert(0, task)
            except StopIteration as si:
                # this will be raised if it is done
                # so we don't put it back on the task list
                # whatever is returned is in the exception's args
                results.append(si.args[0])
            return results
                
print('\n\n*** Running the Loop with fibbonacci numbers\n')

# to use it we create a task loop object and add tasks to it
loop = TaskLoop()
loop.add_task(fib(10))
loop.add_task(fib(5))
# loop.add_task(fib(7))

# let's see how long it takes
start = time.time()
results = loop.run_all()
print(f'total run time: {time.time() - start} seconds')

print('the results are:', results)




*** Running the Loop with fibbonacci numbers


Outer loop count: 1
total run time: 0.0014290809631347656 seconds
the results are: [5]


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:69: RuntimeWarning: coroutine 'fib' was never awaited
